## 1. Install and Import Dependencies

In [1]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121 #pytorch

ERROR: Invalid requirement: '#pytorch'


In [2]:
#some more dependencies
!pip install transformers requests beautifulsoup4 pandas numpy

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests 
from bs4 import BeautifulSoup #our reviews
import re #create a regex function to extract the specific comments we want

## 2. Instantiate the Model

In [4]:
#Our tokenizer and model now loaded and setup
tokenizer= AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model= AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment') 

C:\Users\uruon\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [5]:
#https://huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment

## 3. Encode and Calculate Sentiment
A. Pass through a string or prompt to our tokenizer

B. Tokenize it 

C. And then pass it through to our model and get our classification

In [6]:
uruTokens = tokenizer.encode("To rule over the world, you must rule over yourself first", return_tensors= 'pt')
#passed through a keyword argument to return_tensors and we've set that to py_torch

In [7]:
#For you to test
uruTokens = tokenizer.encode("A mother and daughter were killed by a bad driver", return_tensors= 'pt')
#passed through a keyword argument to return_tensors and we've set that to py_torch

In [8]:
#uruTokens[0], #tokenizer.decode(uruTokens[0]) #a confirmation 

### To perform some sentiment analysis, let's pass this to our model

In [9]:
#Pass our tokens to our model
result = model(uruTokens)
result.logits

tensor([[ 0.8948,  0.4147, -0.0285, -0.4182, -0.8286]],
       grad_fn=<AddmmBackward0>)

In [10]:
#Grab our results and convert them to something useful for us
#torch.argmax(result.logits)

In [11]:
#The highest value represents what position represents the highest sentiment. In  this case, our sentiment score is 4
#As berts base is between 1-5, let's convert to an integer and add 1

In [12]:
int(torch.argmax(result.logits))+1

1

## 4. Collect Reviews

In [13]:
#r= requests.get('https://www.yelp.com/biz/littl e-lagos-atlanta')
#soup = BeautifulSoup(r.text, 'html.parser')
#regex= re.compile('.*comment.*')
#results = soup.find_all('p', {'class':regex}) #using soup.find_all, we're looking for any's with class matching our regex
#always onfirm what it is wrapped in, in this case, wrapped in a <p class= ''comment
#reviews = [result.text for result in results] #List comprehension 

In [14]:
r= requests.get('https://www.yelp.com/biz/southern-suya-atlanta')
soup = BeautifulSoup(r.text, 'html.parser')
regex= re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex}) #using soup.find_all, we're looking for any's with class matching our regex
#always onfirm what it is wrapped in, in this case, wrapped in a <p class= ''comment
reviews = [result.text for result in results] #List comprehension  


In [15]:
#I'll get a response code from my requests.get
#r.text #everything on that page

In [16]:
#Next, we pass this text, to beautiful soup. The next line of code we've written is soup
#Pass r.text into it and then set up our parser
#regex downwards go on to extract the specific things we want from the webpage- extract common pattern of 'comment'
#Then pass the regex to our soup 

In [17]:
#results #all reviews are here- but wrapped in html tags

In [18]:
#to get just text, let's look at say our 12th result
results[7].text

'The joloff rice was good. The fish was not fresh and tasted old. Suya was good and well seasoned. The wait was too long. Nearly an hour.'

In [19]:
for i in range(1, 8):
    print(f"Result {i}: {results[i].text}")


Result 1: If I could give this place more than five stars, I absolutely would! My recent dining experience at this hidden gem was nothing short of extraordinary. From the moment we stepped in, we were greeted with warm smiles and genuine hospitality that made us feel right at home.The staff here went above and beyond to ensure we had a memorable experience. We were a bit indecisive about what to order, but their knowledgeable and friendly server came to our rescue. They enthusiastically recommended a variety of dishes that showcased the best flavors of their culinary offerings.Let's talk about the food; oh my goodness, it was absolutely divine! We took their advice and ordered the beef Suya, Jollof rice, plantains, goat, fried rice, wings, and a side of veggies. Each dish was prepared with meticulous care, bursting with vibrant flavors that had our taste buds dancing in delight.What truly sets this place apart, apart from the exceptional food, is the outstanding customer service. The s

In [20]:
reviews[2]

'The food here was mind blowing. So much flavour in every single item we had. The stew sauce was my favourite.'

## 5. Load Reviews into a DataFrame and Score Them

In [21]:
import numpy as np
import pandas as pd

In [22]:
df= pd.DataFrame(np.array(reviews), columns = ['Review'])

In [23]:
df

,Review
0,I had no idea what beef suya was but I was ama...
1,If I could give this place more than five star...
2,The food here was mind blowing. So much flavou...
3,We were stopping over in Atlanta area for a ni...
4,The food is tasty and has a kick of spice to i...
5,The restaurant is now located at Northlake. Th...
6,The food is amazing. The grill fish and briske...
7,The joloff rice was good. The fish was not fre...
8,They do not do dine in. We came all the way fr...
9,ok peeps check this out.Took some folks from o...


In [24]:
#df.iloc[0]
df['Review'].iloc[0] #Grabbed each review and stored it into our dataframe

"I had no idea what beef suya was but I was amazed at how good it is.  Warning that it's spicy though.  The coconut rice came with a red stew sauce that was delicious as well.  I will definitely go back again.I gave it 5 stars because I like to support food that's beyond the ordinary."

##We're able to take this block of text and pass it to our tokenizer

##Then pass, the tokenizer to our model for Sequence Classification

In [25]:
#Let's define a function to take any of our reviews and output its sentiment
def sentiment_score(anyreview):
    urutokens = tokenizer.encode(anyreview, return_tensors = "pt")
    result = model(urutokens)
    return int(torch.argmax(result.logits))+1

In [26]:
#for an individual indexed review
sentiment_score(df['Review'].iloc[1])

5

In [27]:
#We'll use a lamda function to iterate over all reviews and attach their sentiments
#English- on the dataframe 'Review', take each element and apply our function to it which gives the sentiment
df['sentiment']= df['Review'].apply(lambda x: sentiment_score(x[:512]))
df

,Review,sentiment
0,I had no idea what beef suya was but I was ama...,4
1,If I could give this place more than five star...,5
2,The food here was mind blowing. So much flavou...,5
3,We were stopping over in Atlanta area for a ni...,5
4,The food is tasty and has a kick of spice to i...,5
5,The restaurant is now located at Northlake. Th...,5
6,The food is amazing. The grill fish and briske...,5
7,The joloff rice was good. The fish was not fre...,3
8,They do not do dine in. We came all the way fr...,1
9,ok peeps check this out.Took some folks from o...,3


In [28]:
#let's take a look at individuals
df['Review'].iloc[1]

"If I could give this place more than five stars, I absolutely would! My recent dining experience at this hidden gem was nothing short of extraordinary. From the moment we stepped in, we were greeted with warm smiles and genuine hospitality that made us feel right at home.The staff here went above and beyond to ensure we had a memorable experience. We were a bit indecisive about what to order, but their knowledgeable and friendly server came to our rescue. They enthusiastically recommended a variety of dishes that showcased the best flavors of their culinary offerings.Let's talk about the food; oh my goodness, it was absolutely divine! We took their advice and ordered the beef Suya, Jollof rice, plantains, goat, fried rice, wings, and a side of veggies. Each dish was prepared with meticulous care, bursting with vibrant flavors that had our taste buds dancing in delight.What truly sets this place apart, apart from the exceptional food, is the outstanding customer service. The staff's at